## 2 Background Subtraction

One way to detect movements in a video is to use background subtraction. There are different types of background subtraction algorithm, and each algorithm has various, different parameters. 

This notebook introduces some of background subtractors that are implemented in Birdwatcher, which are based on OpenCV. It shows how to access them, how to check out the various parameters they have, and how they are used for movement detection. Much of what's in here is encaspulated by higher-order functions or classes, but if you want to have full control over things, or better understand what's happening under the hood, have a look at the following.

In [ ]:
import birdwatcher as bw

### Create a video object

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zf20s_low.mp4')

### Create a background subtractor object

Create a background subtractor object with default parameters. This object basically encapsulates the algorithm that determines the background from a history of images, and uses that to find what is not background in the current image. See opencv page for more info on the algorithm: https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html

In [ ]:
bgs = bw.BackgroundSubtractorMOG2()

What are its parameters? In this case all default, because we didn't specify any.

In [ ]:
bgs.get_params()

In the docstrings, you can see the definition of each parameter:

In [ ]:
bw.BackgroundSubtractorMOG2?

You can use non-default parameters by specifying them at intstantiation:

In [ ]:
bgs = bw.BackgroundSubtractorMOG2(VarThreshold=50, NMixtures=8, History=4)

In [ ]:
bgs.get_params()

### Apply background subtractor to video Frames

This can be done by setting up a pipeline that generates and processes image sequences.

First, set up a frame generator that produces gray frames from the color video file object:

In [ ]:
frames = vfs.iter_frames(color=False)

Then, apply to the frames the background subtractor that we created above. It returns another frame generator.

In [ ]:
frames = frames.apply_backgroundsegmenter(bgs, learningRate=-1)

We now have a frame generator that produces foreground mask frames. Let's get rid of some noise. (Look at MorphologyEx page of opencv for what this does: https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html)

In [ ]:
frames = frames.morphologyex(morphtype='open', kernelsize=2)

At this point, none of the methods have actually been run over the frames yet. We just set up a processing pipeline that will run special methods, such as `to video`, which writes the end result to a videofile.

So let's start running the whole pipeline and save results as a video for inspection.

In [ ]:
frames = frames.tovideo('output/test_MOG2.mp4', framerate=vfs.avgframerate)

The creation of the whole pipeline above can also be shortened like so:

In [ ]:
frames = (vfs.iter_frames(color=False)
          .apply_backgroundsegmenter(bgs, learningRate=-1)
          .morphologyex(morphtype='open', kernelsize=2)
          .tovideo('output/test_MOG2.mp4', framerate=vfs.avgframerate))

Let's have a look at the video

In [ ]:
frames.show()

### Another example

Here's another example of a pipeline. We have added a blur manipulation to the videoframes before appying the background segmenter. Also, we have added a region of interest (roi) that tells the background segmenter to only consider pixels within the specified rectangle in the vieo.

In [ ]:
frames = (vfs.iter_frames(color=False)
          .blur((10,10))
          .apply_backgroundsegmenter(bgs, learningRate=-1, roi=(10, 570, 10, 1250))
          .morphologyex(morphtype='open', kernelsize=2)
          .tovideo('output/test2_MOG2.mp4', framerate=vfs.avgframerate))

### Create a coordinate array for reliable storage of the results

For demonstration purposes, we wrote the movement detection results (suprathreshold pixels) above to a video file. However, most video formats are not lossy and cannot always be read efficiently and reliably. Normally you want to save the results for further analyses in a way that is better suited for scientific analyses. We save non-zero pixel (i.e. foreground) coordinates to a `CoordinateArrays` object, including some metadata.

In [ ]:
%%time
coordsarray = (vfs.iter_frames(color=False)
               .apply_backgroundsegmenter(bgs, learningRate=-1)
               .morphologyex(morphtype='open', kernelsize=2)
               .save_nonzero(filepath='output/testcoordsMOG.darr',
                             metadata={'bgsparams': bgs.get_params(),
                                       'morphologyex': ('open', 2),
                                       'learningrate': -1,
                                       'avgframerate': vfs.avgframerate},
                             ignore_firstnframes=10,
                             overwrite=True))

coordsarray

For more information on `CoordinateArrays`, and how to access and view the them, just take a look at the next notebook. Basically it is a disk-based numpy-like array that you can index to retrieve the positive pixel coordinates of frames. E.g., let's  find out which pixels were positive in frame 49:

In [ ]:
coordsarray[49]

### A different algorithm: KNN

Let's try a different algorithm for background segmentation: KNN.  See opencv page for more info on the algorithm: https://docs.opencv.org/3.4/db/d88/classcv_1_1BackgroundSubtractorKNN.html

In [ ]:
bgs = bw.BackgroundSubtractorKNN()
bgs.get_params()

A different background algorithm with different parameters. You can change these parameters the same way as in the example above.

In [ ]:
bgs = bw.BackgroundSubtractorKNN(kNNSamples=0)

And run the whole pipeline again with the other background subtractor, and save the results as video.

In [ ]:
frames = (vfs.iter_frames(color=False)
          .apply_backgroundsegmenter(bgs, learningRate=-1)
          .morphologyex(morphtype='open', kernelsize=2)
          .tovideo('output/test_KNN.mp4', framerate=vfs.avgframerate))

Look at results:

In [ ]:
frames.show()

### Yet another algorithm: LSBP

https://docs.opencv.org/3.4/de/d4c/classcv_1_1bgsegm_1_1BackgroundSubtractorLSBP.html

In [ ]:
bgs = bw.BackgroundSubtractorLSBP()
bgs.get_params()

In [ ]:
bgs = bw.BackgroundSubtractorLSBP(nSamples=10)

In [ ]:
frames = (vfs.iter_frames(color=False)
          .apply_backgroundsegmenter(bgs, learningRate=-1)
          .morphologyex(morphtype='open', kernelsize=2)
          .tovideo('output/test_LSBP.mp4', framerate=vfs.avgframerate))

In [ ]:
frames.show()